In [16]:
import os
import pdfplumber as pp
import pandas as pd

pdf_path_and_name = "crime_report/pdf/crime_report_branch_1_2024.pdf"

pdf = pp.open(pdf_path_and_name)

pages = pdf.pages

tables = []
column_list = []

#총계 리스트
average_list = []

#소계 리스트 
sub_total_list = []

# 데이터 프레임으로 넘기기 위한 메인 카테고리 배열
main_crime_list = []

# 데이터 프레임으로 넘기기 위한 서브 카테고리 배열
sub_crime_list = []

i = 3

for item in pages:
    table = item.extract_table()
    column_list.append(list(filter(None,table[2]))) #컬럼 추출

column_list

[['죄종', '발생건수', '검거건수', '발생대비\n검거건수(%)', '검거인원', '법인체']]

In [25]:
for data in table:
    
    if(i == len(table)):
        break


    # 헤더가 될 데이터 추출

    #소계 헤더
    if(table[i][0] != None and table[i][1] != None):
        sub_total_name = table[i][0] + " "+ f"({table[i][1]})"
        sub_total_죄종 = sub_total_name
    
    # 대분류 헤더
    if(table[i][0] != None): 
        # 0번째 인덱스의 값이 헤더가 되는 경우
        main_row_name = table[i][0]
        main_죄종 = main_row_name
    #소분류  헤더
    elif(table[i][0] == None):
        # 0번째 인덱스가 None 이며 1번째 인덱스의 값이 헤더가 되는 경우
        sub_row_name = table[i][1]
        sub_죄종 = sub_row_name

    #소계 Rows
    if(table[i][0] != None and table[i][1] != None):
        table[i].pop(0)
        table[i].pop(0)

        sub_total_발생건수 = f"{str(table[i][0]).replace(",","")}" 
        sub_total_검거건수 = f"{str(table[i][1]).replace(",","")}"
        sub_total_발생대비검거건수 = table[i][2]
        sub_total_검거인원 = f"{str(table[i][3]).replace(",","")}"
        sub_total_법인체 = f"{str(table[i][4]).replace(",","")}"

        average_list.append([sub_total_죄종,sub_total_발생건수,sub_total_검거건수,sub_total_발생대비검거건수,sub_total_검거인원,sub_total_법인체])
    
    # #대분류
    
    if(table[i][0] != None and table[i][1] == None):       
        table[i].pop(0)
        table[i].pop(0)
        
        main_발생건수 = str(table[i][0]).replace(",","")
        main_검거건수 = str(table[i][1]).replace(",","")
        main_발생대비검거건수 = table[i][2]
        main_검거인원 = str(table[i][3]).replace(",","")
        main_법인체 = str(table[i][4]).replace(",","")

        
        main_crime_list.append([main_죄종,main_발생건수,main_검거건수,main_발생대비검거건수,main_검거인원,main_법인체])
    #소분류
    elif(table[i][0] == None):
         table[i].pop(0)
         table[i].pop(0)

         sub_발생건수 = str(table[i][0]).replace(",","")
         sub_검거건수 = str(table[i][1]).replace(",","")
         sub_발생대비검거건수 = table[i][2]
         sub_검거인원 = str(table[i][3]).replace(",","")
         sub_법인체 = str(table[i][4]).replace(",","")

         sub_crime_list.append([sub_죄종,sub_발생건수,sub_검거건수,sub_발생대비검거건수,sub_검거인원,sub_법인체]) 

    # 인덱스 증가
    i += 1

# main_crime_list 에서 총계데이터 추출 및 sub_total_list 로 재 삽입
average_list.append(main_crime_list[0])
average_list

[['강력범죄 (소계)', '5665', '5393', '95.2', '5678', '1'],
 ['폭력범죄 (소계)', '52973', '46261', '87.3', '59466', '8'],
 ['지능범죄 (소계)', '128560', '65876', '51.2', '65597', '129'],
 ['풍속범죄 (소계)', '5887', '4817', '81.8', '6762', '11'],
 ['총 계', '378908', '278362', '73.5', '300766', '2604'],
 ['총 계', '378908', '278362', '73.5', '300766', '2604']]

In [32]:
# 총합 및 소계 데이터 프레임
average_crime_df = pd.DataFrame(average_list, columns=column_list)

average_crime_df

,죄종,발생건수,검거건수,발생대비\n검거건수(%),검거인원,법인체
0,강력범죄 (소계),5665,5393,95.2,5678,1
1,폭력범죄 (소계),52973,46261,87.3,59466,8
2,지능범죄 (소계),128560,65876,51.2,65597,129
3,풍속범죄 (소계),5887,4817,81.8,6762,11
4,총 계,378908,278362,73.5,300766,2604
5,총 계,378908,278362,73.5,300766,2604


In [4]:
# 대분류 정보 데이터 프레임
main_crime_df = pd.DataFrame(main_crime_list, columns=column_list)

main_crime_df

,죄종,발생건수,검거건수,발생대비\n검거건수(%),검거인원,법인체
0,총 계,378908,278362,73.5,300766,2604
1,절도범죄,44229,29427,66.5,22691,4
2,특별경제범죄,22383,17981,80.3,19450,660
3,마약범죄,2070,1885,91.1,2434,3
4,보건범죄,2423,2355,97.2,3344,56
5,환경범죄,646,606,93.8,689,257
6,교통범죄,56082,54051,96.4,55245,379
7,노동범죄,85,73,85.9,111,13
8,안보범죄,56,49,87.5,110,2
9,선거범죄,101,65,64.4,113,1


In [6]:
# 소분류 범죄 정보 데이터 프레임
sub_crime_df = pd.DataFrame(sub_crime_list, columns=column_list)
sub_crime_df

,죄종,발생건수,검거건수,발생대비\n검거건수(%),검거인원,법인체
0,살인기수,73,71,97.3,81,0
1,살인미수등,112,110,98.2,118,1
2,강도,118,118,100,172,0
3,강간,1230,1188,96.6,1288,0
4,유사강간,225,213,94.7,229,0
5,강제추행,3608,3419,94.8,3497,0
6,기타 강간/강제추행등,39,38,97.4,52,0
7,방화,260,236,90.8,241,0
8,상해,4926,4763,96.7,5761,0
9,폭행,27511,26532,96.4,34499,2


In [7]:
sub_main_mixed_crime_df = pd.concat([main_crime_df, sub_crime_df], ignore_index=True)
total_crime_df = pd.concat([sub_main_mixed_crime_df, average_crime_df], ignore_index=True)

In [9]:
average_crime_df.to_csv('crime_report/2024/average/crime_report_branch_1_2024_average_crime_data.csv', index=False)
sub_crime_df.to_csv('crime_report/2024/main/crime_report_branch_1_2024_sub_crime_data.csv', index=False)
main_crime_df.to_csv('crime_report/2024/sub/crime_report_branch_1_2024_main_crime_data.csv', index=False)
total_crime_df.to_csv('crime_report/2024/tatal/crime_report_branch_1_2024_tatal_crime_data.csv', index=False)